In [15]:
import yaml, joblib
from evml.model import DNN
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GroupShuffleSplit

In [3]:
mlp_config = "/glade/work/schreck/repos/evidential/evidential/results/mlp/model.yml"
with open(mlp_config) as cf:
    mlp_conf = yaml.load(cf, Loader=yaml.FullLoader)

In [7]:
save_loc = mlp_conf["save_loc"]
input_cols = mlp_conf["input_cols"]
output_cols = mlp_conf["output_cols"]
input_size = len(input_cols)
middle_size = mlp_conf["middle_size"]
output_size = mlp_conf["output_size"]
batch_size = mlp_conf["batch_size"]
dropout = mlp_conf["dropout"]
batch_norm = mlp_conf["batch_norm"]

### Load some data

In [11]:
data = pd.read_csv(mlp_conf["idaho"])

In [14]:
data["day"] = data["Time"].apply(lambda x: str(x).split(" ")[0])
flat_seed = 1000

# Need the same test_data for all trained models (data and model ensembles)
gsp = GroupShuffleSplit(n_splits=1,  random_state = flat_seed, train_size=0.9)
splits = list(gsp.split(data, groups = data["day"]))
train_index, test_index = splits[0]
train_data, test_data = data.iloc[train_index].copy(), data.iloc[test_index].copy() 

# Make N train-valid splits using day as grouping variable
gsp = GroupShuffleSplit(n_splits=100,  random_state = flat_seed, train_size=0.885)
splits = list(gsp.split(train_data, groups = train_data["day"]))
train_index, valid_index = splits[data_seed]
train_data, valid_data = train_data.iloc[train_index].copy(), train_data.iloc[valid_index] .copy()  

### Load pre-trained x-scaler and model 

In [16]:
with open(f"{save_loc}/scalers.pkl", "rb") as fid:
    scaler_x, scaler_y = joblib.load(fid)

In [19]:
x_train = scaler_x.transform(train_data[input_cols])
x_valid = scaler_x.transform(valid_data[input_cols])
x_test  = scaler_x.transform(test_data[input_cols])

### Load pre-trained MLP

In [17]:
device = "cuda"

In [23]:
model = DNN(
    input_size, 
    output_size, 
    block_sizes = [middle_size], 
    dr = [dropout], 
    batch_norm = batch_norm, 
    lng = False
).to(device)

_ model.load_weights(f"{save_loc}/best.pt")

0

### Predict with the model

In [25]:
x_splits = [x_train, x_valid, x_test]
splits = [train_data, valid_data, test_data]
        
for x,y in zip(x_splits, splits):
    y_pred = model.predict(x, batch_size = batch_size)
    y["y_pred"] = scaler_y.inverse_transform(y_pred)

### Next up the evidential regression model

In [26]:
save_loc = "/glade/work/schreck/repos/evidential/evidential/results/evmlp"

In [31]:
with open(f"{save_loc}/scalers.pkl", "rb") as fid:
    scaler_x, scaler_y = joblib.load(fid)

In [32]:
x_train = scaler_x.transform(train_data[input_cols])
x_valid = scaler_x.transform(valid_data[input_cols])
x_test  = scaler_x.transform(test_data[input_cols])

In [33]:
model = DNN(
    input_size, 
    output_size, 
    block_sizes = [middle_size], 
    dr = [dropout], 
    batch_norm = batch_norm, 
    lng = True
).to(device)

_ = model.load_weights(f"{save_loc}/best.pt")

##### Compute variance in the training outputs

In [34]:
sigma = train_data[output_cols[0]].std()

In [36]:
x_splits = [x_train, x_valid, x_test]
splits = [train_data, valid_data, test_data]
        
for x,y in zip(x_splits, splits):
    y_pred = model.predict(x, batch_size = batch_size)              
    y["mu"] = scaler_y.inverse_transform(y_pred[:,0:1])
    y["v"] = y_pred[:,1:2]
    y["alpha"] = y_pred[:,2:3]
    y["beta"] = y_pred[:,3:4]

    inverse_evidence = 1. / ((y["alpha"].copy()-1) * y["v"])
    variance = y["beta"] * inverse_evidence

    y["var"] = (variance * sigma**2)
    y["conf"] = (inverse_evidence * sigma**2)